In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [7]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40
path = "../data/ModelNet40"
categories = []
tr, te = get_dataloaders(path, categories=categories)

Loading renders for airplane:  27%|█████████████████████████████████████▍                                                                                                       | 166/626 [00:14<00:40, 11.27it/s]


OSError: image file is truncated

In [3]:
from my_models.spvd import SPVUnet

down_blocks = [{
        "features_list": [32, 64, 128, 192, 192, 256],
        "num_layers_list": 1,
        "attn_heads": (None, None, None, 8, 8)
    }]
    up_blocks = [
        {
            "features_list": [256, 192, 192],
            "num_layers_list": 2,
            "attn_heads": 8
        },
        {
            "features_list": [192, 128, 64, 32],
            "num_layers_list": 2,
            "attn_heads": None
        },
    ]
    t_emb_features = 64

    model = SPVUnet(down_blocks, up_blocks, t_emb_features)
    model = models.DiffusionBase(model, lr=1e-4)

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

lr = 1e-4
model = models.DiffusionBase(model, lr=lr)

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [4]:
import lightning as L
trainer = L.Trainer(
    max_epochs=40,
    gradient_clip_val=10.0,
)

trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 15.0 M | train
------------------------------------------
15.0 M    Trainable params
0         Non-trainable params
15.0 M    Total params
60.100    Total estimated model params size (MB)
289       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/opt/conda/envs/spvd/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=40` reached.


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

preds = ddpm_sched.sample(model.cuda().eval(), 16, 2048)

from utils.visualization import visualize_notebook
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)

 39%|██████████████████████████████████████████████████████████████████▎                                                                                                       | 390/1000 [00:33<00:52, 11.53it/s]


RuntimeError: The size of tensor a (32710) must match the size of tensor b (32712) at non-singleton dimension 0

In [ ]:
import models

m = models.SPVD()

lr = 1e-4
model2 = models.DiffusionBase(m, lr=lr)

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("../logs/ModelNet40/bottle2", name="test")

checkpoint_name = "-".join(categories) if categories else "all"
epochs = 40
checkpoint_callback = ModelCheckpoint(
    dirpath=f'../checkpoints/ModelNet40/{checkpoint_name}',
    filename="{epoch:02d}-{val_loss:.2f}", 
    every_n_epochs=5
)

trainer = L.Trainer(
    logger=logger,
    log_every_n_steps=5,
    max_epochs=epochs, 
    gradient_clip_val=10.0, 
    callbacks=[checkpoint_callback]
)

In [5]:
trainer.fit(model=model2, train_dataloaders=tr, val_dataloaders=te)

You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/envs/spvd/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ubuntu/SPVD_Lightning/checkpoints/ModelNet40/bottle exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 32.9 M | train
------------------------------------------
32.9 M    Trainable params
0         Non-trainable params
32.9 M    Total params
131.576   Total estimated model params size (MB)
314       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=40` reached.


In [6]:
torch.cuda.empty_cache()

In [7]:
from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

preds = ddpm_sched.sample(model2.cuda().eval(), 16, 2048)

from utils.visualization import visualize_notebook
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)

 62%|█████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                 | 618/1000 [00:37<00:23, 16.56it/s]


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
